# Second experiment: awake / n-rem / rem

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from keras import layers
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import random as random

trainset_1 = pd.read_csv('EEG_mouse_data_1.csv')
trainset_2 = pd.read_csv('EEG_mouse_data_2.csv')
testset_1 = pd.read_csv('EEG_mouse_data_test.csv')

In [10]:
# normlisation des features
from sklearn.preprocessing import StandardScaler
import pandas as pd

def normalize(df):
    headers = df.columns
    scaler = StandardScaler()
    
    numerical_columns = df.select_dtypes(include=['int', 'float']).columns
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    normalized_data = pd.DataFrame(df, columns=headers[:-1])

    return normalized_data

In [11]:
# extraction des colonnes
# envisager de faire la séléction des features ici
def extract_features(data, n):
    return data.iloc[:,:n+1]


In [12]:
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from keras import layers
from sklearn.model_selection import KFold

def create_model_three_class():
  # 2 inputs
  # 2 hidden nodes
  # 1 output

  mlp = keras.Sequential([
      layers.Input(shape=(25,)),
      layers.Dense(3, activation="tanh"), # Try different numbers of hidden neurons here (e.g. 2, 4, 8, 32, 128)
      layers.Dense(3, activation="tanh"),
  ])

  # Experiment with hyperparameters here:
  # momentum: [0, 0.8, 0.9, 0.99]
  # learning_rate: [0.1, 0.01, 0.001, 0.0001]
  mlp.compile(
      optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.99),
      loss="mse",
  )

  return mlp
mlp = create_model_three_class()
mlp.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 3)                   │              78 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 3)                   │              12 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 90 (360.00 B)

 Trainable params: 90 (360.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 3 classes

import seaborn as sns
import matplotlib.pyplot as plt
import keras
from keras import layers
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import random as random

# Programme principal de traitement du train set #1
normal_trainset_1_three_class = extract_features(trainset_1, 26)
normal_trainset_1_three_class = normalize(normal_trainset_1_three_class)
normal_trainset_1_three_class = normal_trainset_1_three_class.sample(frac=1, random_state=42)

# splitting en 3 folds
keras.utils.set_random_seed(123)
kf = KFold(n_splits=3, shuffle=True)

y = normal_trainset_1_three_class['state'].map({'w': 2, 'r': 1, 'n': 0})
normal_trainset_1_three_class.drop(columns=['state'], inplace=True)

history_list_three_class = []
trained_mlp_three_class = []
for fold, (train_index, test_index) in enumerate(kf.split(normal_trainset_1_three_class)):
    
    model = create_model_three_class()
    
    history = model.fit(x=normal_trainset_1_three_class.iloc[train_index], y=y.iloc[train_index], 
                       validation_data=(normal_trainset_1_three_class.iloc[test_index], y.iloc[test_index]),
                       epochs=50)
    
    history_list_three_class.append(history)
    trained_mlp_three_class.append(model)

Epoch 1/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.9868 - val_loss: 0.7082
Epoch 2/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6878 - val_loss: 0.7056
Epoch 3/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6823 - val_loss: 0.7045
Epoch 4/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6808 - val_loss: 0.7044
Epoch 5/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6793 - val_loss: 0.7036
Epoch 6/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6784 - val_loss: 0.7037
Epoch 7/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6775 - val_loss: 0.7042
Epoch 8/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6770 - val_loss: 0.7052
Epoch 9/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6763 - val_loss: 0.7059
Epoch 10/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6757 - val_loss: 0.7063
Epoch 11/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6757 - val_loss: 0.7079
Epoch 12/50
423/423 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

In [ ]:
# classes

import numpy as np
import matplotlib.pyplot as pl
train_losses = np.array([history.history['loss'] for history in history_list_three_class])
val_losses = np.array([history.history['val_loss'] for history in history_list_three_class])

# Calculate mean and standard deviation for training and validation losses
mean_train_loss = np.mean(train_losses, axis=0)
std_train_loss = np.std(train_losses, axis=0)
mean_val_loss = np.mean(val_losses, axis=0)
std_val_loss = np.std(val_losses, axis=0)

# Plot mean and standard deviation for training loss
pl.plot(mean_train_loss, label='Training Loss (Mean)')
pl.fill_between(range(len(mean_train_loss)), mean_train_loss - std_train_loss, mean_train_loss + std_train_loss, alpha=0.3, label='Training Loss (Std)')

# Plot mean and standard deviation for validation loss
pl.plot(mean_val_loss, label='Validation Loss (Mean)')
pl.fill_between(range(len(mean_val_loss)), mean_val_loss - std_val_loss, mean_val_loss + std_val_loss, alpha=0.3, label='Validation Loss (Std)')

# Add labels and legend
pl.xlabel('Epochs')
pl.ylabel('Loss')
pl.legend()

# Display the plot
pl.show()

In [ ]:
# 3 classes

import numpy as np
from sklearn.metrics import confusion_matrix, f1_score
import seaborn as sns

def plot_confusion_matrix(confusion_matrix, title):
    # Plot confusion matrix
    pl.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix.astype(int), annot=True, fmt="d", cmap="Blues", cbar=False,
                xticklabels=["n", "r", "w"], yticklabels=["n", "r", "w"])
    pl.title(title)
    pl.xlabel('Predicted')
    pl.ylabel('True')
    pl.show()

f1_scores = []
mean_confusion_matrix = np.zeros((3, 3))

for i, (train_index, test_index) in enumerate(kf.split(normal_trainset_1_three_class)):
    # Evaluate the trained model on the test fold
    predicted_probabilities = trained_mlp_three_class[i].predict(normal_trainset_1_three_class.iloc[test_index])
    predictions = np.argmax(predicted_probabilities, axis=1)
    true_labels = y.iloc[test_index]

    # Compute confusion matrix
    cm = confusion_matrix(true_labels, predictions)
    mean_confusion_matrix += confusion_matrix(true_labels, predictions)

    # Compute confusion matrix and plot
    #plot_confusion_matrix(cm, f'Confusion Matrix - Fold {i + 1}')

    # Compute F1 score
    f1 = f1_score(true_labels, predictions, average='micro')
    f1_scores.append(f1)
    print(f"F1 Score - Fold {i + 1}: {f1}")

# Plot mean confusion matrix
plot_confusion_matrix(mean_confusion_matrix, 'Global confusion matrix')

# Calculate and display the mean F1 score across all folds
mean_f1_score = np.mean(f1_scores)
print(f"Mean F1 Score across all folds: {mean_f1_score}")